In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Activation, GRU
from keras.layers import Dropout, SpatialDropout1D
from keras.layers import Bidirectional,TimeDistributed, concatenate
from keras.layers import GlobalMaxPool1D, GlobalAvgPool1D, Masking
from keras.models import Model,Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import TensorBoard
from keras import metrics

In [ ]:
import pandas as pd
import numpy as np
import scipy
import sys

In [ ]:
from process_data import load_data,get_rank_of_size
from layers import AttentionLayer,MyMeanPool
from data_util import unpack_news_data,DATE_INTERVAL_NEWS,MAX_NEWS_NUM,EMBEDDING_SIZE

In [ ]:
rank = get_rank_of_size()

In [ ]:
data = load_data(rank[:5])

In [ ]:
data['GOOGL']['train'][0][0][0].toarray().shape

# model

In [ ]:
def build_baseline_model(code='Default'):
    news_input = Input(shape=(DATE_INTERVAL_NEWS,MAX_NEWS_NUM,EMBEDDING_SIZE))
    inv_layer = GRU(100,return_sequences=True)
    
    x = news_input
    x = TimeDistributed(Masking(mask_value=0.))(x)
    x = TimeDistributed(SpatialDropout1D(0.1))(x)
    x = TimeDistributed(AttentionLayer())(x)
    
    x = inv_layer(x)
    x = SpatialDropout1D(0.2)(x)
    x = AttentionLayer()(x)
    
    x = Dense(2, activation='softmax')(x)
    model = Model(inputs=news_input,outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
model = build_baseline_model()

In [ ]:
print(model.summary())

In [ ]:
for i in range(5):
    model.fit(x=unpack_news_data(data[rank[i]]['train'][0]),y=data[rank[i]]['train'][2],
            batch_size=64,epochs=3,verbose=1,
            validation_data=(unpack_news_data(data[rank[i]]['val'][0]),data[rank[i]]['val'][2]),
            callbacks=[TensorBoard(log_dir='model_log/')])

In [ ]:
model.evaluate(x=unpack_news_data(data[rank[0]]['val'][0]), y=data[rank[0]]['val'][2],batch_size=64)

# NUMERICAL MODEL

In [ ]:
numerical_timestep = 20 #  correspond to the 'size' of  the window
attribute_num = 5 # Open/High/Low/AdjClose/Volume 

def build_numerical_model(code='Default'):
    numerical_input = Input(shape=(numerical_timestep,attribute_num))
    x = GRU(50,return_sequences=True)(numerical_input)
    x = Dropout(0.5)(x)
    x = GRU(50)(x)
    x = Dropout(0.5)(x)
    x = Dense(10)(x)
    x = Dense(2,activation='softmax')(x)
    model = Model(inputs=numerical_input,outputs=x)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
nmodel = build_numerical_model()

In [ ]:
print(nmodel.summary())

In [ ]:
nmodel.fit(x=data[rank[0]]['train'][1],y=data[rank[0]]['train'][2],batch_size=16,epochs=100,verbose=2,\
          validation_data=(data[rank[0]]['val'][1],data[rank[0]]['val'][2]),\
          callbacks=[TensorBoard(log_dir='num_log/')])